In [ ]:
import numpy as np
import suite2p
# from tempfile import TemporaryDirectory

### suite2p

In [ ]:
ops = suite2p.default_ops()
ops['save_NWB'] = True
print(ops)

db = {
    'data_path': ['./data/'],
    'save_path0': './data/',
    'tiff_list': ['Sue_2x_3000_40_-46.tif'],
}
db

output_ops = suite2p.run_s2p(ops=ops, db=db)

In [ ]:
from pathlib import Path

output_op = output_ops
print('param:\n', set(output_op.keys()).difference(ops.keys()))
print('\noutput path: \n', list(Path(output_op['save_path']).iterdir()))

In [ ]:
import h5py
import os
# extension = os.path.splitext(file_path)[1].lower()

f = h5py.File('./data/suite2p/ophys.nwb', "r")

def PrintOnlyDataset(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(name)

with h5py.File('./data/suite2p/ophys.nwb', "r") as f:
    f.visititems(PrintOnlyDataset)

In [ ]:
f = h5py.File('./data/suite2p/ophys.nwb', "r")
print(f['acquisition']['TwoPhotonSeries']['starting_time'])
print(f['processing']['ophys']['Backgrounds_0']['meanImg'])

In [ ]:
from pynwb import NWBHDF5IO
f = NWBHDF5IO('./data/suite2p/ophys.nwb', 'r')
nwbfile = f.read()
nwbfile

### NWB format
- acquisition
    - TwoPhotonSeries
- processing
    - ophys
- imageing_planes
    - ImagePlane

In [ ]:
# from suite2p.io.nwb import read_nwb
# read_nwb('./data/suite2p/ophys.nwb')

In [ ]:
from natsort import natsorted 
plane_folders = natsorted([ f.path for f in os.scandir('./data/suite2p') if f.is_dir() and f.name[:5]=='plane'])
ops1 = [np.load(join_file_path(f, 'ops.npy'), allow_pickle=True).item() for f in plane_folders]
ops = ops1[0]

In [ ]:
from pynwb import NWBFile
from pynwb.ophys import (
    OpticalChannel, TwoPhotonSeries, ImageSegmentation,
    RoiResponseSeries, Fluorescence
)

In [ ]:
### INITIALIZE NWB FILE
nwbfile = NWBFile(
    session_description='suite2p_proc',
    identifier='./data',
    session_start_time=(ops['date_proc'] if 'date_proc' in ops else datetime.datetime.now())  # 現在時刻
)
print(nwbfile)

### aquisition

In [ ]:
# 顕微鏡情報を登録
device = nwbfile.create_device(
    name='Microscope',
    description='My two-photon microscope',   # 顕微鏡の情報
    manufacturer='The best microscope manufacturer'
)

# 光チャネルの説明を登録
optical_channel = OpticalChannel(
    name='OpticalChannel', 
    description='an optical channel', 
    emission_lambda=500.
)

In [ ]:
imaging_plane = nwbfile.create_imaging_plane(
    name='ImagingPlane',
    description='standard',
    optical_channel=optical_channel,   # 光チャネル
    device=device,   # 電極デバイス
    imaging_rate=ops['fs'],   # 画像の比率Hz
    excitation_lambda=600., # 励起（れいき）波長
    indicator='GCaMP',   # カルシウムインディケーター
    location='V1',
    grid_spacing=([2.0,2.0]),   # Space between pixels in (x, y)
    grid_spacing_unit='microns'
)

In [ ]:
image_series = TwoPhotonSeries(
    name='TwoPhotonSeries', 
    dimension=[ops['Ly'], ops['Lx']],
    external_file=['./data/Sue_2x_3000_40_-46.tif'], 
    imaging_plane=imaging_plane,
    format='external',   # Three types: 1) Image format; tiff, png, jpg, etc. 2) external 3) raw.
    starting_time=0.0, 
    starting_frame=[0], 
    rate=ops['fs'] * ops['nplanes']   #  Sampling rate in Hz
)

In [ ]:
nwbfile.add_acquisition(image_series)

### ophys/ImageSegmentation

In [ ]:
nwbfile.create_processing_module(
    name='ophys', 
    description='optical physiology processed data'
)
img_seg = ImageSegmentation()
nwbfile.processing['ophys'].add(img_seg)

#### ophys/ImageSegmentation/PlaneSegmentation

In [ ]:
plane_segmentation = img_seg.create_plane_segmentation(
    name='PlaneSegmentation',
    description='suite2p output',
    imaging_plane=imaging_plane,
    reference_images=image_series
)

#### PlaneSegmentationにROIとIscellを追加

In [ ]:
# iscell
iscell = np.load(join_file_path(ops['save_path'], 'iscell.npy'))

# stat
stat = np.load(join_file_path(ops['save_path'], 'stat.npy'), allow_pickle=True)

# roi
ncells = len(stat)
roi_list = []

for i in range(ncells):
#     plane_segmentation.add_roi(pixel_mask=pixel_mask.T)
    kargs = {}
    kargs['pixel_mask'] = np.array([
        stat[i]['ypix'], stat[i]['xpix'], stat[i]['lam']]).T
    roi_list.append(kargs)

# for col in roi_list[0].keys():
#     # import pdb; pdb.set_trace()
#     if col not in plane_segmentation.colnames:
#         plane_segmentation.add_column(col, f'{col} list')
    
for col in roi_list:
    plane_segmentation.add_roi(**col)

ncells_all = 0
ncells_all += ncells

In [ ]:
plane_segmentation.columns

In [ ]:
plane_segmentation.columns[1].data

In [ ]:
# plane_segmentation.add_column('iscell', 'two columns - iscell & probcell', iscell)

### FLUORESCENCE, Neuropil, Deconvolvedを登録

In [ ]:
# # ROI
# region_roi = plane_segmentation.create_roi_table_region(
#     region=list(np.arange(0, ncells_all)),
#     description='all ROIs'
# )

In [ ]:
file_strs = ['F.npy', 'Fneu.npy', 'spks.npy']
name_strs = ['Fluorescence', 'Neuropil', 'Deconvolved']
# file_strs = ['F.npy']
# name_strs = ['Fluorescence']

for i, (fstr,nstr) in enumerate(zip(file_strs, name_strs)):
    data = np.load(join_file_path(ops['save_path'], fstr)) 
    region_roi = plane_segmentation.create_roi_table_region(
        nstr,
        region=list(np.arange(ncells_all)),
    )
    roi_resp_series = RoiResponseSeries(
        name=nstr,
        data=data,
        rois=region_roi,
        unit='lumens',
        rate=ops['fs']
    )
    fluo = Fluorescence(
        roi_response_series=roi_resp_series,
        name=nstr
    )
    print(roi_resp_series.data.shape)
    nwbfile.processing['ophys'].add(fluo)

In [ ]:
nwbfile.processing['ophys'].data_interfaces

In [ ]:
nwbfile.processing['ophys'].data_interfaces['Fluorescence'].roi_response_series['Fluorescence'].data.shape


In [ ]:
nwbfile.processing['ophys'].data_interfaces['Neuropil'].roi_response_series['Neuropil'].data.shape

In [ ]:
nwbfile.processing['ophys']

In [ ]:
nwbfile.processing['ophys']

### backgrounds

In [ ]:
# from pynwb.base import Images
# from pynwb.image import GrayscaleImage

# # BACKGROUNDS
# # (meanImg, Vcorr and max_proj are REQUIRED)
# bg_strs = ['meanImg', 'Vcorr', 'max_proj', 'meanImg_chan2']
# nplanes = ops['nplanes']

# images = Images(f'Backgrounds_{0}')
# for bstr in bg_strs:
#     if bstr in ops:
#         if bstr=='Vcorr' or bstr=='max_proj':
#             img = np.zeros((ops['Ly'], ops['Lx']), np.float32)
#             img[ops['yrange'][0]:ops['yrange'][-1], 
#                 ops['xrange'][0]:ops['xrange'][-1]] = ops[bstr]
#         else:
#             img = ops[bstr]
#         images.add_image(GrayscaleImage(name=bstr, data=img))

In [ ]:
# nwbfile.processing['ophys'].add(images)

In [ ]:
with NWBHDF5IO(join_file_path('./data', 'ophys.nwb'), 'w') as fio:
    fio.write(nwbfile)

In [ ]:
images

In [ ]:
def PrintOnlyDataset(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(name)

with h5py.File('./data/suite2p/ophys.nwb', "r") as f:
    f.visititems(PrintOnlyDataset)